In [165]:
import my_funcs as mf
import pandas as pd
import verde as vd
import numpy as np
import geopandas as gpd
import rioxarray

import pyproj

import matplotlib.pyplot as plt
gdb = '/home/ggrl/geodatabase/'

In [166]:
projection = pyproj.Proj(proj='utm', zone='23s', EPGS ='32723')

In [35]:
malha_cartog_1kk = mf.importar('malha_cartog_1kk_wgs84')

In [46]:
malha_cartog_50k_1089 = gpd.read_file('/home/ggrl/Desktop/1089_area_50k.shp')

In [36]:
malha_cartog_1kk.head(10)

,id_folha,geometry
0,SB18,"POLYGON ((-78.00000 -4.00000, -72.00000 -4.000..."
1,SC18,"POLYGON ((-78.00000 -8.00000, -72.00000 -8.000..."
2,NA19,"POLYGON ((-72.00000 4.00000, -66.00000 4.00000..."
3,SA19,"POLYGON ((-72.00000 0.00000, -66.00000 0.00000..."
4,SB19,"POLYGON ((-72.00000 -4.00000, -66.00000 -4.000..."
5,SC19,"POLYGON ((-72.00000 -8.00000, -66.00000 -8.000..."
6,NB20,"POLYGON ((-66.00000 8.00000, -60.00000 8.00000..."
7,NA20,"POLYGON ((-66.00000 4.00000, -60.00000 4.00000..."
8,SA20,"POLYGON ((-66.00000 0.00000, -60.00000 0.00000..."
9,SB20,"POLYGON ((-66.00000 -4.00000, -60.00000 -4.000..."


In [58]:
malha_cartog_50k_1089.head(6)

,geometry,region,id_folha
0,"POLYGON ((-45.25000 -10.25000, -45.00000 -10.2...","(-45.25, -45.0, -10.5, -10.25)",SC23_Y_B_III_4
1,"POLYGON ((-45.25000 -10.50000, -45.00000 -10.5...","(-45.25, -45.0, -10.75, -10.5)",SC23_Y_B_VI_2
2,"POLYGON ((-45.25000 -10.75000, -45.00000 -10.7...","(-45.25, -45.0, -11.0, -10.75)",SC23_Y_B_VI_4
3,"POLYGON ((-45.25000 -11.00000, -45.00000 -11.0...","(-45.25, -45.0, -11.25, -11.0)",SC23_Y_D_III_2
4,"POLYGON ((-45.00000 -10.25000, -44.75000 -10.2...","(-45.0, -44.75, -10.5, -10.25)",SC23_Z_A_I_3
5,"POLYGON ((-45.00000 -10.50000, -44.75000 -10.5...","(-45.0, -44.75, -10.75, -10.5)",SC23_Z_A_IV_1


In [50]:
malha_cartog_50k_1089.drop(['region','id_folha','field_1'],axis=1,inplace=True)

In [52]:
# Definindo Regions (W,E,S,N)
bounds = malha_cartog_50k_1089.bounds

malha_cartog_50k_1089['region'] = [(left,right,bottom,top) for left,right,bottom,top in zip(bounds['minx'],bounds['maxx'],
                                                                                            bounds['miny'],bounds['maxy'])]

In [90]:
malha_cartog_50k_1089.head(5)

,geometry,region,id_folha
0,"POLYGON ((-45.25000 -10.25000, -45.00000 -10.2...","(-45.25, -45.0, -10.5, -10.25)",SC23_Y_B_III_4
1,"POLYGON ((-45.25000 -10.50000, -45.00000 -10.5...","(-45.25, -45.0, -10.75, -10.5)",SC23_Y_B_VI_2
2,"POLYGON ((-45.25000 -10.75000, -45.00000 -10.7...","(-45.25, -45.0, -11.0, -10.75)",SC23_Y_B_VI_4
3,"POLYGON ((-45.25000 -11.00000, -45.00000 -11.0...","(-45.25, -45.0, -11.25, -11.0)",SC23_Y_D_III_2
4,"POLYGON ((-45.00000 -10.25000, -44.75000 -10.2...","(-45.0, -44.75, -10.5, -10.25)",SC23_Z_A_I_3


In [91]:
df = pd.DataFrame(malha_cartog_50k_1089)

In [55]:
# Definindo nomes a partir da articulação sistematica de folhas de cartas. Construindo uma lista e definindo como uma series.
lista_grid=[]

for index, row in df.iterrows():
    row['id_folha'] = (mf.nomeador_grid(row.region[0],row.region[1],
                                        row.region[3],row.region[2]))
    lista_grid.append(row.id_folha)

malha_cartog_50k_1089['id_folha'] = lista_grid

In [56]:
type(malha_cartog_1kk['id_folha'])

pandas.core.series.Series

In [57]:
malha_cartog_50k_1089.head(5)

,geometry,region,id_folha
0,"POLYGON ((-45.25000 -10.25000, -45.00000 -10.2...","(-45.25, -45.0, -10.5, -10.25)",SC23_Y_B_III_4
1,"POLYGON ((-45.25000 -10.50000, -45.00000 -10.5...","(-45.25, -45.0, -10.75, -10.5)",SC23_Y_B_VI_2
2,"POLYGON ((-45.25000 -10.75000, -45.00000 -10.7...","(-45.25, -45.0, -11.0, -10.75)",SC23_Y_B_VI_4
3,"POLYGON ((-45.25000 -11.00000, -45.00000 -11.0...","(-45.25, -45.0, -11.25, -11.0)",SC23_Y_D_III_2
4,"POLYGON ((-45.00000 -10.25000, -44.75000 -10.2...","(-45.0, -44.75, -10.5, -10.25)",SC23_Z_A_I_3


In [101]:
df.region

0     (-45.25, -45.0, -10.5, -10.25)
1     (-45.25, -45.0, -10.75, -10.5)
2     (-45.25, -45.0, -11.0, -10.75)
3     (-45.25, -45.0, -11.25, -11.0)
4     (-45.0, -44.75, -10.5, -10.25)
5     (-45.0, -44.75, -10.75, -10.5)
6     (-45.0, -44.75, -11.0, -10.75)
7     (-45.0, -44.75, -11.25, -11.0)
8     (-44.75, -44.5, -10.25, -10.0)
9     (-44.75, -44.5, -10.5, -10.25)
10    (-44.75, -44.5, -10.75, -10.5)
11    (-44.75, -44.5, -11.0, -10.75)
12     (-44.5, -44.25, -10.0, -9.75)
13    (-44.5, -44.25, -10.25, -10.0)
14    (-44.5, -44.25, -10.5, -10.25)
15    (-44.5, -44.25, -10.75, -10.5)
16    (-44.5, -44.25, -11.0, -10.75)
17     (-44.25, -44.0, -10.0, -9.75)
18    (-44.25, -44.0, -10.25, -10.0)
19    (-44.25, -44.0, -10.5, -10.25)
20    (-44.25, -44.0, -10.75, -10.5)
21    (-44.25, -44.0, -11.0, -10.75)
22      (-44.0, -43.75, -9.75, -9.5)
23     (-44.0, -43.75, -10.0, -9.75)
24    (-44.0, -43.75, -10.25, -10.0)
25    (-44.0, -43.75, -10.5, -10.25)
26    (-44.0, -43.75, -10.75, -10.5)
2

In [104]:
for index, row in df.iterrows():
    print(row.region[0])
    break

-45.25


In [114]:
g_1105 = pd.read_csv(gdb+'geof/g1105')
geof_1039 = pd.read_csv(gdb+'geof/geof_1039')
g_1089 = pd.read_csv(gdb+'geof/g1089')

In [115]:
g_1089.columns

Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')

In [187]:
chain_list = ['CTCOR','KPERC','eU','eth','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

for index, row in df.iterrows():

    print(row.id_folha+' start')
    data = g_1089[vd.inside((g_1089.LONGITUDE, g_1089.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    
    for i in chain_list:

        chain = vd.Chain([
            ('trend', vd.Trend(degree=1)),
            ('reduce', vd.BlockReduce(np.median, spacing=1000)),
            ('spline', vd.Spline())
        ])
        
        chain.fit(coordinates, data[i])
        
        grid = chain.grid(spacing=200, data_names=[i],pixel_register=True)
        grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)
        tif_ = grids[i].rename(easting = 'x',northing='y')
        tif_.rio.to_raster('/home/ggrl/Desktop/grids/'+i+'_'+row.id_folha+'.tif')

SC23_Y_B_III_4 start
20296 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Y_B_III_4 end
SC23_Y_B_VI_2 start
20374 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Y_B_VI_2 end
SC23_Y_B_VI_4 start
20558 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Y_B_VI_4 end
SC23_Y_D_III_2 start
19950 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Y_D_III_2 end
SC23_Z_A_I_3 start
20583 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Z_A_I_3 end
SC23_Z_A_IV_1 start
20626 Index(['X', 'Y', 'MDT', 'KPERC', 'eU', 'eth', 'CTCOR', 'LONGITUDE',
       'LATITUDE'],
      dtype='object')
CTCOR
SC23_Z_A_IV_1 end
SC23_Z_A_IV_3 start
20611 Index(['X', 

In [188]:
chain_list = ['CTCOR','KPERC','eU','eth','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

for index, row in df.iterrows():

    print(row.id_folha+' start')
    data = g_1089[vd.inside((g_1089.LONGITUDE, g_1089.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    
    for i in chain_list:

        chain = vd.Chain([
            ('trend', vd.Trend(degree=1)),
            ('reduce', vd.BlockReduce(np.median, spacing=1000)),
            ('spline', vd.Spline())
        ])
        
        chain.fit(coordinates, data[i])
        
        grid = chain.grid(spacing=200, data_names=[i],pixel_register=True)
        grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)
        tif_ = grids[i].rename(easting = 'x',northing='y')
        tif_.rio.to_raster('/home/ggrl/Desktop/grids/'+i+'_'+row.id_folha+'.tif')
print('end')

SC23_Y_B_III_4 start
SC23_Y_B_VI_2 start
SC23_Y_B_VI_4 start
SC23_Y_D_III_2 start
SC23_Z_A_I_3 start
SC23_Z_A_IV_1 start
SC23_Z_A_IV_3 start
SC23_Z_C_I_1 start
SC23_Z_A_I_2 start
SC23_Z_A_I_4 start
SC23_Z_A_IV_2 start
SC23_Z_A_IV_4 start
SC23_X_C_V_3 start
SC23_Z_A_II_1 start
SC23_Z_A_II_3 start
SC23_Z_A_V_1 start
SC23_Z_A_V_3 start
SC23_X_C_V_4 start
SC23_Z_A_II_2 start
SC23_Z_A_II_4 start
SC23_Z_A_V_2 start
SC23_Z_A_V_4 start
SC23_X_C_VI_1 start
SC23_X_C_VI_3 start
SC23_Z_A_III_1 start
SC23_Z_A_III_3 start
SC23_Z_A_VI_1 start
SC23_Z_A_VI_3 start
SC23_X_C_VI_2 start
SC23_X_C_VI_4 start
SC23_Z_A_III_2 start
SC23_Z_A_III_4 start
SC23_Z_A_VI_2 start
SC23_Z_A_VI_4 start


In [ ]:
chain_list = ['CTCOR','KPERC','eU','eth','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

for index, row in df.iterrows():

    print(row.id_folha+' start')
    data = g_1089[vd.inside((g_1089.LONGITUDE, g_1089.LATITUDE), region = row.region)]
    coordinates = (data.X.values, data.Y.values)
    
    for i in chain_list:

        chain = vd.Chain([
            ('trend', vd.Trend(degree=1)),
            ('reduce', vd.BlockReduce(np.median, spacing=1000)),
            ('spline', vd.Spline())
        ])
        
        chain.fit(coordinates, data[i])
        
        grid = chain.grid(spacing=200, data_names=[i],pixel_register=True)
        grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)
        tif_ = grids[i].rename(easting = 'x',northing='y')
        tif_.rio.to_raster('/home/ggrl/Desktop/grids/'+i+'_'+row.id_folha+'.tif')
print('end')

In [ ]:
'''
    cv     = vd.BlockKFold(spacing=1000,
                n_splits=3,
                shuffle=True)

    scores[i] = vd.cross_val_score(chain,
                            coordinates,
                            data[i],
                            cv=cv)
'''

In [193]:
l_100k = mf.importar('l_100k')
f_rparaim = mf.mapa(l_100k,'Rio Paraim')
f_rparaim.unary_union.bounds

(-45.00000000000001,
 -11.000000000000036,
 -44.500000000000014,
 -10.499999999999957)

In [195]:
data = g_1089[vd.inside((g_1089.LONGITUDE, g_1089.LATITUDE), region = [-45.0,-44.5,
                                                                      -11.0,-10.5])]
coordinates = (data.X.values, data.Y.values)

In [197]:
chain_list = ['CTCOR','KPERC','eU','eth','MDT']

grids = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

scores = {'CTCOR':(),'KPERC':(), 'eU':(), 'eth':(), 'MDT':()}

for i in chain_list:

        chain = vd.Chain([
            ('trend', vd.Trend(degree=1)),
            ('reduce', vd.BlockReduce(np.median, spacing=1000)),
            ('spline', vd.Spline())
        ])
        
        chain.fit(coordinates, data[i])
        
        grid = chain.grid(spacing=200, data_names=[i],pixel_register=True)
        grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)
        tif_ = grids[i].rename(easting = 'x',northing='y')
        tif_.rio.to_raster('/home/ggrl/Desktop/grids/'+i+'_rio_paraim.tif')
print('end')

end
